In [ ]:
pip install stop-words


  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=58aae4ebef4085f403bd440fe3ced75ffc9ec3ec7b47db5464d3702519a8f90c
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [ ]:
pip install spacy


In [ ]:

import numpy as np
import pandas as pd

from stop_words import get_stop_words # Procesado
import unicodedata
import re # Procesado
import string
import nltk
nltk.download('punkt')# Procesado
nltk.download('wordnet')# Procesado
import spacy# Procesado


from nltk.tokenize import sent_tokenize #Procesado

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve # Reporte
from nltk.stem import WordNetLemmatizer # Procesado
import matplotlib.pyplot as plt # Reporte

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Pipeline

In [ ]:
# Crear un lematizador
nlp = spacy.load("en_core_web_sm")

# Caracteres numéricos
numeric_pattern = re.compile(r'\b\d+\b')

# Palabras a eliminar
play_game_words = {'play', 'game', 'games', 'toy', 'one'}

def text_normalizer(text, lemmatize=False, sw_list=None):

    processed_text = []

    if sw_list is None:
        sw_list = get_stop_words('en') #lista de stopwords en inglés
    # Convierte el texto a minúsuculas
    text = text.lower()
    # Eliminar caracteres "extraños"
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # Tabla para eliminar signos de puntuación
    table = str.maketrans('', '', string.punctuation)
    # Separar las frases
    sentences = sent_tokenize(text)

    for sentence in sentences:
        words = sentence.split(' ')
        for word in words:
            if word not in string.punctuation and word not in sw_list and not numeric_pattern.match(word) and word not in play_game_words: #quitamos signos de puntuacion numericos, stopwords,palabras seleccionadas
                word = word.translate(table)
                processed_text.append(word)

    processed_text = ' '.join(processed_text)

    if lemmatize:
        lemmatized_text = ' '.join([token.lemma_ for token in nlp(processed_text)]) #lematizamos
        return lemmatized_text
    else:
        return processed_text


#Cargamos el data

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/data_file.csv", sep=';')

In [ ]:
df = pd.DataFrame.from_dict(data)

In [ ]:
df.head()

,overall,reviewText,overall_label
0,1.0,Disapointed! I purchased this product because...,1
1,1.0,This does not fit the ATW Cargo Wagon and shou...,1
2,1.0,Parts missing,1
3,1.0,This product box was in awful condition. It l...,1
4,1.0,"Very very cheaply made, the pot tilts already ...",1


Eliminar Espacios al inicio y al final

In [ ]:
df['reviewText'] = df['reviewText'].str.strip()

In [ ]:
df['reviewText'] = df['reviewText'].apply(lambda x: text_normalizer(x, lemmatize=True))

In [ ]:
# Visualizar los resultados
df.head()

,overall,reviewText,overall_label
0,1.0,disapointe purchase product picture descriptio...,1
1,1.0,fit atw cargo wagon list accessory it pad smal...,1
2,1.0,part miss,1
3,1.0,product box awful condition looked smash rip t...,1
4,1.0,cheaply make pot tilt already without bean who...,1


In [ ]:
df['reviewText'].get(10)

'buy horse love daughter christmas couple year ago fun fall apart just day felt totally rip off stable idea great pretend daughter pretty upset buy wooden modelsnbspa datahookproductlinklinke classalinknormal hrefmaximlargestablewithcorraldpb00078anhurefcmcrarpdrvwtxtieutf8maxim large stable corralanbsplast year worth extra money still great shape daughter still play regularly'

Despues de normalizar el texto es posible que si en alguna review solo aparecia stopword y/o alguna de las palabras qhe he eliminado podría haber un na, lo comprobamos y si es así borramos esos registros

In [ ]:
# Contar los valores perdidos en las columnas 'overall' y 'reviewText'
na_counts = df[['overall', 'reviewText']].isna().sum()

# Imprimir el número de valores perdidos en cada columna
print("Número de valores perdidos en 'overall' y 'reviewText':")
print(na_counts)

Número de valores perdidos en 'overall' y 'reviewText':
overall       0
reviewText    0
dtype: int64


In [ ]:
df.dropna(subset=['overall', 'reviewText'], inplace=True)
df.reset_index(drop=True, inplace=True)

#crear csv


In [ ]:
df.to_csv('data_file_processed.csv', index=False,sep=';')

# Subir el archivo CSV a Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp data_file_processed.csv /content/drive/MyDrive/data_file_processed.csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
